In [1]:
import pandas as pd
import os
from glob import glob
from reports import get_average_classification_report
pd.options.display.float_format = '{:.2f}'.format
from functools import reduce
pd.options.display.float_format = '{:.2f}'.format

In [2]:
root_dir = '/media/dmlab/My Passport/DATA/ComBERT'
filepaths = sorted(glob(os.path.join(root_dir, 'result_*', 'classification_report_train_*.csv')))
dirnames = sorted(list(set([os.path.dirname(filepath) for filepath in filepaths])))
dirnames

['/media/dmlab/My Passport/DATA/ComBERT/result_bert-base-uncased',
 '/media/dmlab/My Passport/DATA/ComBERT/result_finbert',
 '/media/dmlab/My Passport/DATA/ComBERT/result_finbert-pretrain',
 '/media/dmlab/My Passport/DATA/ComBERT/result_pt_bert-base-uncased_with_company_masking_first=True',
 '/media/dmlab/My Passport/DATA/ComBERT/result_pt_bert-base-uncased_wo_company_masking_first=None',
 '/media/dmlab/My Passport/DATA/ComBERT/result_pt_finbert-pretrain_with_company_masking_first=True',
 '/media/dmlab/My Passport/DATA/ComBERT/result_pt_finbert-pretrain_wo_company_masking_first=None',
 '/media/dmlab/My Passport/DATA/ComBERT/result_pt_finbert_with_company_masking_first=True',
 '/media/dmlab/My Passport/DATA/ComBERT/result_pt_finbert_wo_company_masking_first=None',
 '/media/dmlab/My Passport/DATA/ComBERT/result_pt_sec-bert-base_with_company_masking_first=True',
 '/media/dmlab/My Passport/DATA/ComBERT/result_pt_sec-bert-base_wo_company_masking_first=None',
 '/media/dmlab/My Passport/DATA/

# Result
* `yiyanghkust/finbert-pretrain`이 가장 좋은 성능을 보임
> finbert-pretrain의 사전 학습 데이터: Corporate Reports 10-K & 10-Q: 2.5B tokens, Earnings Call Transcripts: 1.3B tokens, Analyst Reports: 1.1B tokens

* `Company name masking`을 적용할 때 성능이 향상됨

In [8]:
def get_df(filepaths, colname):
    records = []
    for report_filepath in filepaths:
        mode = os.path.basename(report_filepath).split('_')[-1].replace('.csv', '')
        df = pd.read_csv(report_filepath).set_index('Unnamed: 0')
        acc = 100*df.filter(items = ['accuracy'], axis=0)['f1-score'].values[0]
        records.append((int(mode), acc))
    return pd.DataFrame(records, columns=['trained_with', colname]).sort_values(by=['trained_with'])

dfs = []
for dirname in dirnames:
    filepaths = glob(os.path.join(dirname, 'classification_report_*.csv'))
    filepaths = [item for item in filepaths if int(item.split('_')[-1].replace('.csv', ''))>=200]
# #     filepaths = [item for item in filepaths if int(item.split('_')[-1].replace('.csv', ''))<40000]
    name = os.path.basename(dirname.replace('result_', ''))
    df = get_df(filepaths, name)    
    dfs.append(df)    
result = reduce(lambda df1,df2: pd.merge(df1,df2,on='trained_with'), dfs)
result.set_index('trained_with', inplace=True)
result.loc['Average'] = result.mean()

dict_to_rename_columns = {'bert-base-uncased': 'BERT', 'finbert': 'FinBERT-Araci', 'finbert-pretrain': 'FinBERT-Yang',
       'pt_bert-base-uncased_with_company_masking_first=True': 'BERT_pt_CM',
       'pt_bert-base-uncased_wo_company_masking_first=None': 'BERT_pt_subword',
       'pt_finbert-pretrain_with_company_masking_first=True': 'FinBERT-Yang_pt_CM',
       'pt_finbert-pretrain_wo_company_masking_first=None': 'FinBERT-Yang_pt_subword',
       'pt_finbert_with_company_masking_first=True': 'FinBERT-Araci_pt_CM',
       'pt_finbert_wo_company_masking_first=None': 'FinBERT-Araci_pt_subword',
       'pt_sec-bert-base_with_company_masking_first=True': 'SEC-BERT_pt_CM',
       'pt_sec-bert-base_wo_company_masking_first=None': 'SEC-BERT_pt_subword', 'sec-bert-base': 'SEC-BERT'}
new_colnames = [dict_to_rename_columns[item] for item in result.columns]
result.columns = new_colnames

filepath = os.path.join(root_dir, 'classification_report.csv')
result.to_csv(filepath)
print('Created {}'.format(filepath))

result

Created /media/dmlab/My Passport/DATA/ComBERT/classification_report.csv


,BERT,FinBERT-Araci,FinBERT-Yang,BERT_pt_CM,BERT_pt_subword,FinBERT-Yang_pt_CM,FinBERT-Yang_pt_subword,FinBERT-Araci_pt_CM,FinBERT-Araci_pt_subword,SEC-BERT_pt_CM,SEC-BERT_pt_subword,SEC-BERT
trained_with,,,,,,,,,,,,
200,55.70,17.27,50.60,59.57,49.00,64.06,52.94,59.25,53.83,64.52,59.60,64.50
300,60.24,32.20,47.54,56.21,49.51,63.59,53.54,56.74,59.11,59.75,59.15,66.16
400,62.26,41.49,48.54,67.98,50.74,72.70,58.82,60.21,60.96,67.41,62.61,60.86
500,66.64,50.45,60.76,72.54,71.24,77.39,73.45,75.84,70.23,73.06,69.50,68.20
600,66.39,52.91,68.82,76.39,62.08,76.13,72.67,79.77,61.99,76.24,69.02,69.79
700,64.16,62.61,68.51,75.79,70.64,78.35,74.44,79.63,74.94,73.75,71.96,70.58
800,64.76,64.65,69.96,70.76,74.69,79.02,76.29,80.42,68.07,76.87,73.04,69.41
900,71.68,70.11,71.13,78.54,79.83,80.29,79.17,80.43,77.94,78.57,73.14,69.08
1000,73.14,73.41,74.50,77.32,80.26,80.63,78.74,80.47,79.77,80.37,74.29,74.72


* `Company name masking`의 성능 향상 효과는 finetuning 데이터가 적을 때 특히 더 효과적으로 작용함

In [9]:
low = result.filter(items = [200, 300, 400, 500, 600, 700, 800, 900], axis=0)
low.loc['Average'] = low.mean()

filepath = os.path.join(root_dir, 'classification_report_Low.csv')
low.to_csv(filepath)
print('Created {}'.format(filepath))

low

Created /media/dmlab/My Passport/DATA/ComBERT/classification_report_Low.csv


,BERT,FinBERT-Araci,FinBERT-Yang,BERT_pt_CM,BERT_pt_subword,FinBERT-Yang_pt_CM,FinBERT-Yang_pt_subword,FinBERT-Araci_pt_CM,FinBERT-Araci_pt_subword,SEC-BERT_pt_CM,SEC-BERT_pt_subword,SEC-BERT
trained_with,,,,,,,,,,,,
200,55.70,17.27,50.60,59.57,49.00,64.06,52.94,59.25,53.83,64.52,59.60,64.50
300,60.24,32.20,47.54,56.21,49.51,63.59,53.54,56.74,59.11,59.75,59.15,66.16
400,62.26,41.49,48.54,67.98,50.74,72.70,58.82,60.21,60.96,67.41,62.61,60.86
500,66.64,50.45,60.76,72.54,71.24,77.39,73.45,75.84,70.23,73.06,69.50,68.20
600,66.39,52.91,68.82,76.39,62.08,76.13,72.67,79.77,61.99,76.24,69.02,69.79
700,64.16,62.61,68.51,75.79,70.64,78.35,74.44,79.63,74.94,73.75,71.96,70.58
800,64.76,64.65,69.96,70.76,74.69,79.02,76.29,80.42,68.07,76.87,73.04,69.41
900,71.68,70.11,71.13,78.54,79.83,80.29,79.17,80.43,77.94,78.57,73.14,69.08
Average,63.98,48.96,60.73,69.72,63.47,73.94,67.67,71.54,65.88,71.27,67.25,67.32
